In [2]:
import os
import time
import pandas as pd
import requests

#### Get OSM cafe features

In [4]:
cities = pd.read_excel('cities.xlsx',header=None,names=['city'])

In [5]:
cities["city_name"] = cities["city"].str.split(",").str[0].str.strip()
cities["state_code"] = cities["city"].str.split(",").str[1].str.strip()

cities.head()

,city,city_name,state_code
1,"New York, NY",New York,NY
2,"Los Angeles, CA",Los Angeles,CA
3,"Chicago, IL",Chicago,IL
4,"Houston, TX",Houston,TX
5,"Phoenix, AZ",Phoenix,AZ


In [6]:
OVERPASS_URL = "https://overpass-api.de/api/interpreter"

def run_overpass_query(query, timeout=180, max_retries=5):
    for attempt in range(max_retries):
        try:
            r = requests.post(OVERPASS_URL, data={"data": query}, timeout=timeout+30)
            r.raise_for_status()
            return r.json()
        except requests.exceptions.HTTPError as e:
            status = e.response.status_code
            # transient / rate-limit / overload errors
            if status in (429, 502, 503, 504):
                wait = 3 * (attempt + 1)
                print(f"    Overpass HTTP {status} on attempt {attempt+1}, sleeping {wait}s then retrying...")
                time.sleep(wait)
                continue
            else:
                print(f"    Fatal HTTP error {status}: {e}")
                break
        except requests.exceptions.RequestException as e:
            # network glitches, timeouts, etc
            wait = 3 * (attempt + 1)
            print(f"    Request error on attempt {attempt+1}: {e}. Sleeping {wait}s then retrying...")
            time.sleep(wait)
            continue

    print("    Giving up on this query, returning empty result.")
    return {"elements": []}

In [7]:
def get_osm_cafes(city_name, state_code, timeout=180):
    query = f"""
    [out:json][timeout:{timeout}];
    area["name"="{city_name}"]["boundary"="administrative"]["admin_level"~"6|7|8"]->.a;

    (
      node["amenity"="cafe"](area.a);
      way["amenity"="cafe"](area.a);
      relation["amenity"="cafe"](area.a);

      node["amenity"="fast_food"]["cuisine"~"coffee|coffee_shop"](area.a);
      way["amenity"="fast_food"]["cuisine"~"coffee|coffee_shop"](area.a);
      relation["amenity"="fast_food"]["cuisine"~"coffee|coffee_shop"](area.a);

      node["name"~"Starbucks|Dunkin|Peet|Tim Hortons"](area.a);
      way["name"~"Starbucks|Dunkin|Peet|Tim Hortons"](area.a);
      relation["name"~"Starbucks|Dunkin|Peet|Tim Hortons"](area.a);
    );

    out center tags;
    """

    data = run_overpass_query(query, timeout=timeout)
    els = data.get("elements", [])
    rows = []

    for el in els:
        tags = el.get("tags", {}) or {}

        if el["type"] == "node":
            lat, lon = el.get("lat"), el.get("lon")
        else:
            c = el.get("center") or {}
            lat, lon = c.get("lat"), c.get("lon")

        rows.append({
            "name": tags.get("name"),
            "brand": tags.get("brand"),
            "amenity": tags.get("amenity"),
            "cuisine": tags.get("cuisine"),
            "lat": lat,
            "lon": lon,
            "city_name": city_name,
            "state_code": state_code,
        })

    return pd.DataFrame(rows)

In [8]:
FEATURE_PATH = "osm_city_cafe_features.csv"

In [9]:
# Build list of (city_name, state_code) pairs from your cities df
city_pairs = list(zip(cities["city_name"], cities["state_code"]))

In [10]:
# If this is run before, load previous results and resume
if os.path.exists(FEATURE_PATH):
    city_features = pd.read_csv(FEATURE_PATH)

    # Use both city_name and state_code as key
    done_pairs = set(zip(city_features["city_name"], city_features["state_code"]))
    print(f"Loaded {len(done_pairs)} previously processed city/state pairs.")
else:
    city_features = pd.DataFrame()
    done_pairs = set()

In [11]:
for city_name, state_code in city_pairs:
    key = (city_name, state_code)
    if key in done_pairs:
        print(f"Skipping already processed city: {city_name}, {state_code}")
        continue

    print(f"Processing {city_name}, {state_code}...")

    # Fetch cafe data
    try:
        df_cafe = get_osm_cafes(city_name, state_code)
    except Exception as e:
        print(f"  Error getting cafes for {city_name}, {state_code}: {e}")
        df_cafe = pd.DataFrame()

    # Count
    n_cafe = len(df_cafe)

    if n_cafe == 0:
        print(f"  WARNING: No cafes found for {city_name}, {state_code}")

    # Build base feature dict
    features = {
        "city_name": city_name,
        "state_code": state_code,
        "n_cafe": int(n_cafe),
    }

    # -------------------- brand counts --------------------
    if n_cafe > 0 and "brand" in df_cafe.columns:
        brands = df_cafe["brand"].dropna().astype(str).str.strip()
        brand_counts = brands.value_counts()

        for brand, count in brand_counts.items():
            col = f"n_cafe_brand_{brand.replace(' ', '_')}"
            features[col] = int(count)

    # -------------------- amenity counts --------------------
    if n_cafe > 0 and "amenity" in df_cafe.columns:
        amenities = df_cafe["amenity"].dropna().astype(str).str.strip()
        amenity_counts = amenities.value_counts()

        for amenity, count in amenity_counts.items():
            col = f"n_cafe_amenity_{amenity.replace(' ', '_')}"
            features[col] = int(count)

    # -------------------- cuisine counts --------------------
    if n_cafe > 0 and "cuisine" in df_cafe.columns:
        cuisines = (
            df_cafe["cuisine"]
            .dropna()
            .astype(str)
            .str.split(";")
            .explode()
            .str.strip()
        )
        cuisine_counts = cuisines.value_counts()

        for cuisine, count in cuisine_counts.items():
            col = f"n_cafe_cuisine_{cuisine.replace(' ', '_')}"
            features[col] = int(count)

    # -------------------- Save row --------------------
    row_df = pd.DataFrame([features])
    city_features = pd.concat([city_features, row_df], ignore_index=True)
    city_features = city_features.fillna(0)

    city_features.to_csv(FEATURE_PATH, index=False)

    print(
        f"  - Retrieved {n_cafe} cafes "
        f"(Saved features for {city_name}, {state_code}. Current shape: {city_features.shape})"
    )
    
    # nice to Overpass
    time.sleep(1)


Processing New York, NY...
  - Retrieved 0 cafes (Saved features for New York, NY. Current shape: (1, 3))
Processing Los Angeles, CA...
  - Retrieved 800 cafes (Saved features for Los Angeles, CA. Current shape: (2, 112))
Processing Chicago, IL...
  - Retrieved 954 cafes (Saved features for Chicago, IL. Current shape: (3, 142))
Processing Houston, TX...
  - Retrieved 313 cafes (Saved features for Houston, TX. Current shape: (4, 162))
Processing Phoenix, AZ...
  - Retrieved 255 cafes (Saved features for Phoenix, AZ. Current shape: (5, 173))
Processing Philadelphia, PA...
  - Retrieved 372 cafes (Saved features for Philadelphia, PA. Current shape: (6, 181))
Processing San Antonio, TX...
  - Retrieved 159 cafes (Saved features for San Antonio, TX. Current shape: (7, 183))
Processing San Diego, CA...
  - Retrieved 494 cafes (Saved features for San Diego, CA. Current shape: (8, 196))
Processing Dallas, TX...
  - Retrieved 152 cafes (Saved features for Dallas, TX. Current shape: (9, 199))
Pr

#### Get NYC cafes

In [64]:
def get_nyc_cafes(timeout=180):
    nyc_aliases = ["Manhattan", "Brooklyn", "Queens", "Bronx", "Staten Island"]
    all_rows = []

    cafe_query_template = """
    [out:json][timeout:{timeout}];
    area["name"="{borough}"]["boundary"="administrative"]["admin_level"~"6|7|8"]->.a;

    (
      node["amenity"="cafe"](area.a);
      way["amenity"="cafe"](area.a);
      relation["amenity"="cafe"](area.a);

      node["amenity"="fast_food"]["cuisine"~"coffee|coffee_shop"](area.a);
      way["amenity"="fast_food"]["cuisine"~"coffee|coffee_shop"](area.a);
      relation["amenity"="fast_food"]["cuisine"~"coffee|coffee_shop"](area.a);

      node["name"~"Starbucks|Dunkin|Peet|Tim Hortons"](area.a);
      way["name"~"Starbucks|Dunkin|Peet|Tim Hortons"](area.a);
      relation["name"~"Starbucks|Dunkin|Peet|Tim Hortons"](area.a);
    );

    out center tags;
    """

    for borough in nyc_aliases:
        print(f"Querying NYC borough: {borough}")

        query = cafe_query_template.format(timeout=timeout, borough=borough)
        data = run_overpass_query(query, timeout=timeout)
        els = data.get("elements", [])

        for el in els:
            tags = el.get("tags", {}) or {}

            if el["type"] == "node":
                lat = el.get("lat")
                lon = el.get("lon")
            else:
                c = el.get("center") or {}
                lat = c.get("lat")
                lon = c.get("lon")

            all_rows.append({
                "name": tags.get("name"),
                "brand": tags.get("brand"),
                "amenity": tags.get("amenity"),
                "cuisine": tags.get("cuisine"),
                "lat": lat,
                "lon": lon,
                "borough": borough,
                "city_name": "New York",
                "state_code": "NY",
            })

    # Convert to DataFrame
    df = pd.DataFrame(all_rows)

    # Remove duplicates if any
    if not df.empty:
        df = df.drop_duplicates(subset=["name", "lat", "lon"])

    return df

In [66]:
nyc_aliases = ["Manhattan", "Brooklyn", "Queens", "Bronx", "Staten Island"]

FEATURE_PATH = "osm_city_cafe_features_new_york.csv"

# If this is run before, load previous results and resume
if os.path.exists(FEATURE_PATH):
    city_features = pd.read_csv(FEATURE_PATH)
    done_pairs = set(zip(city_features["city_name"], city_features["state_code"]))
    print(f"Loaded {len(done_pairs)} previously processed city/state pairs.")
else:
    city_features = pd.DataFrame()
    done_pairs = set()

# ----------------- New York only -----------------
city_name = "New York"
state_code = "NY"
key = (city_name, state_code)

if key in done_pairs:
    print(f"Skipping already processed city: {city_name}, {state_code}")
else:
    print(f"Processing {city_name}, {state_code} via boroughs...")

    # Fetch cafes for each borough and combine
    cafe_frames = []
    for borough in nyc_aliases:
        print(f"    Getting cafes for borough: {borough}...")
        try:
            df_cafe_b = get_osm_cafes(borough, state_code)
        except Exception as e:
            print(f"      Error getting cafes for {borough}, {state_code}: {e}")
            df_cafe_b = pd.DataFrame()

        n_b = len(df_cafe_b)
        print(f"      -> {n_b} cafes found for {borough}")
        if n_b > 0:
            cafe_frames.append(df_cafe_b)

    if cafe_frames:
        df_cafe = pd.concat(cafe_frames, ignore_index=True)
    else:
        df_cafe = pd.DataFrame()

    # Total cafes across all boroughs
    n_cafe = len(df_cafe)
    if n_cafe == 0:
        print(f"  WARNING: No cafes found for {city_name}, {state_code}")

    # Build base feature dict
    features = {
        "city_name": city_name,
        "state_code": state_code,
        "n_cafe": int(n_cafe),
    }

    # -------------------- brand counts --------------------
    if n_cafe > 0 and "brand" in df_cafe.columns:
        brands = df_cafe["brand"].dropna().astype(str).str.strip()
        brand_counts = brands.value_counts()

        for brand, count in brand_counts.items():
            col = f"n_cafe_brand_{brand.replace(' ', '_')}"
            features[col] = int(count)

    # -------------------- amenity counts --------------------
    if n_cafe > 0 and "amenity" in df_cafe.columns:
        amenities = df_cafe["amenity"].dropna().astype(str).str.strip()
        amenity_counts = amenities.value_counts()

        for amenity, count in amenity_counts.items():
            col = f"n_cafe_amenity_{amenity.replace(' ', '_')}"
            features[col] = int(count)

    # -------------------- cuisine counts --------------------
    if n_cafe > 0 and "cuisine" in df_cafe.columns:
        cuisines = (
            df_cafe["cuisine"]
            .dropna()
            .astype(str)
            .str.split(";")
            .explode()
            .str.strip()
        )
        cuisine_counts = cuisines.value_counts()

        for cuisine, count in cuisine_counts.items():
            col = f"n_cafe_cuisine_{cuisine.replace(' ', '_')}"
            features[col] = int(count)

    # -------------------- Save row --------------------
    row_df = pd.DataFrame([features])
    city_features = pd.concat([city_features, row_df], ignore_index=True)
    city_features = city_features.fillna(0)

    city_features.to_csv(FEATURE_PATH, index=False)

    print(
        f"  - Retrieved {n_cafe} cafes "
        f"(Saved features for {city_name}, {state_code}. Current shape: {city_features.shape})"
    )

Processing New York, NY via boroughs...
    Getting cafes for borough: Manhattan...
      -> 1350 cafes found for Manhattan
    Getting cafes for borough: Brooklyn...
    Overpass HTTP 429 on attempt 1, sleeping 3s then retrying...
    Overpass HTTP 429 on attempt 2, sleeping 6s then retrying...
    Overpass HTTP 429 on attempt 3, sleeping 9s then retrying...
    Overpass HTTP 429 on attempt 4, sleeping 12s then retrying...
      -> 871 cafes found for Brooklyn
    Getting cafes for borough: Queens...
      -> 536 cafes found for Queens
    Getting cafes for borough: Bronx...
      -> 0 cafes found for Bronx
    Getting cafes for borough: Staten Island...
    Overpass HTTP 429 on attempt 1, sleeping 3s then retrying...
    Overpass HTTP 429 on attempt 2, sleeping 6s then retrying...
      -> 103 cafes found for Staten Island
  - Retrieved 2860 cafes (Saved features for New York, NY. Current shape: (1, 191))


In [95]:
city_features

,city_name,state_code,n_cafe,n_cafe_brand_Dunkin',n_cafe_brand_Starbucks,n_cafe_brand_Le_Pain_Quotidien,n_cafe_brand_Gong_Cha,n_cafe_brand_Joe_&_The_Juice,n_cafe_brand_Kung_Fu_Tea,n_cafe_brand_Blue_Bottle_Coffee,...,n_cafe_cuisine_eastern_european,n_cafe_cuisine_pie,n_cafe_cuisine_pancake,n_cafe_cuisine_,n_cafe_cuisine_austrian,n_cafe_cuisine_kebab,n_cafe_cuisine_bangladeshi,n_cafe_cuisine_panini,n_cafe_cuisine_cannabis,n_cafe_cuisine_argentinian
0,New York,NY,2860,373,269,24,19,16,16,16,...,1,1,1,1,1,1,1,1,1,1


#### Merge and clean

In [128]:
df = pd.read_csv('osm_city_cafe_features.csv')
df = df.drop_duplicates()
df.head()

,city_name,state_code,n_cafe,n_cafe_brand_Starbucks,n_cafe_brand_The_Coffee_Bean_&_Tea_Leaf,n_cafe_brand_Dunkin',n_cafe_brand_Peet's_Coffee,n_cafe_brand_Blue_Bottle_Coffee,n_cafe_brand_It's_Boba_Time,n_cafe_brand_Le_Pain_Quotidien,...,n_cafe_cuisine_honeybar,n_cafe_brand_Tropical_Smoothie_Cafe,n_cafe_brand_Space_Rabbit_Coffee,n_cafe_brand_Bar_Hashery,n_cafe_brand_Cooplands,n_cafe_cuisine_nepalese,n_cafe_brand_Wells_Fargo,n_cafe_cuisine_farm_store,n_cafe_brand_Axum,n_cafe_cuisine_donair
0,New York,NY,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Los Angeles,CA,800,173.0,37.0,18.0,7.0,7.0,5.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Chicago,IL,954,133.0,0.0,187.0,8.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Houston,TX,313,102.0,0.0,8.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Phoenix,AZ,255,79.0,3.0,15.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [130]:
# Identify the New York row in df
mask_df = (df["city_name"] == "New York")  # add & (df["state_code"] == "NY") if you have it

# Identify the New York row in city_features
mask_cf = (city_features["city_name"] == "New York")  # and state_code if present

# Sanity check: both should be True exactly once
assert mask_df.sum() == 1, "New York should appear exactly once in df"
assert mask_cf.sum() == 1, "New York should appear exactly once in city_features"

# Get the overlapping columns (only columns that exist in BOTH)
shared_cols = [c for c in df.columns if c in city_features.columns and c not in ["city_name", "state_code"]]

# Extract the one-row slices for New York (aligned on shared_cols)
df_ny = df.loc[mask_df, shared_cols]
cf_ny = city_features.loc[mask_cf, shared_cols]

# Build a boolean mask: where df is 0 and city_features is not 0
overwrite_mask = (df_ny == 0) & (cf_ny != 0)

# Apply: keep df_ny unless overwrite_mask is True, then take cf_ny
df.loc[mask_df, shared_cols] = df_ny.where(~overwrite_mask, cf_ny)
df.head()

,city_name,state_code,n_cafe,n_cafe_brand_Starbucks,n_cafe_brand_The_Coffee_Bean_&_Tea_Leaf,n_cafe_brand_Dunkin',n_cafe_brand_Peet's_Coffee,n_cafe_brand_Blue_Bottle_Coffee,n_cafe_brand_It's_Boba_Time,n_cafe_brand_Le_Pain_Quotidien,...,n_cafe_cuisine_honeybar,n_cafe_brand_Tropical_Smoothie_Cafe,n_cafe_brand_Space_Rabbit_Coffee,n_cafe_brand_Bar_Hashery,n_cafe_brand_Cooplands,n_cafe_cuisine_nepalese,n_cafe_brand_Wells_Fargo,n_cafe_cuisine_farm_store,n_cafe_brand_Axum,n_cafe_cuisine_donair
0,New York,NY,2860,269.0,1.0,373.0,1.0,16.0,0.0,24.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,Los Angeles,CA,800,173.0,37.0,18.0,7.0,7.0,5.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Chicago,IL,954,133.0,0.0,187.0,8.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Houston,TX,313,102.0,0.0,8.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Phoenix,AZ,255,79.0,3.0,15.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [132]:
# Identify cuisine and brand columns
cuisine_cols = [c for c in df.columns if c.startswith("n_cafe_cuisine_")]
brand_cols   = [c for c in df.columns if c.startswith("n_cafe_brand_")]

print("Cuisine columns:", len(cuisine_cols))
print("Brand columns:", len(brand_cols))

# Count how many cuisine columns are > 0
df["n_cafe_cuisine"] = (df[cuisine_cols] > 0).sum(axis=1)

# Count how many brand columns are > 0
df["n_cafe_brand"] = (df[brand_cols] > 0).sum(axis=1)

df.head()

Cuisine columns: 395
Brand columns: 321


,city_name,state_code,n_cafe,n_cafe_brand_Starbucks,n_cafe_brand_The_Coffee_Bean_&_Tea_Leaf,n_cafe_brand_Dunkin',n_cafe_brand_Peet's_Coffee,n_cafe_brand_Blue_Bottle_Coffee,n_cafe_brand_It's_Boba_Time,n_cafe_brand_Le_Pain_Quotidien,...,n_cafe_brand_Space_Rabbit_Coffee,n_cafe_brand_Bar_Hashery,n_cafe_brand_Cooplands,n_cafe_cuisine_nepalese,n_cafe_brand_Wells_Fargo,n_cafe_cuisine_farm_store,n_cafe_brand_Axum,n_cafe_cuisine_donair,n_cafe_cuisine,n_cafe_brand
0,New York,NY,2860,269.0,1.0,373.0,1.0,16.0,0.0,24.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,115,53
1,Los Angeles,CA,800,173.0,37.0,18.0,7.0,7.0,5.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,78,29
2,Chicago,IL,954,133.0,0.0,187.0,8.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,43,28
3,Houston,TX,313,102.0,0.0,8.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,44,18
4,Phoenix,AZ,255,79.0,3.0,15.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25,12


In [134]:
threshold = 0.9
keep_cols = (df == 0).mean() < threshold

df_reduced = df.loc[:, keep_cols]

print("Original columns:", df.shape[1])
print("Remaining columns:", df_reduced.shape[1])

Original columns: 733
Remaining columns: 23


In [136]:
df_reduced.columns

Index(['city_name', 'state_code', 'n_cafe', 'n_cafe_brand_Starbucks',
       'n_cafe_brand_Dunkin'', 'n_cafe_amenity_cafe',
       'n_cafe_amenity_fast_food', 'n_cafe_cuisine_coffee_shop',
       'n_cafe_cuisine_bubble_tea', 'n_cafe_cuisine_donut',
       'n_cafe_cuisine_sandwich', 'n_cafe_cuisine_breakfast',
       'n_cafe_cuisine_american', 'n_cafe_cuisine_coffee',
       'n_cafe_cuisine_pastry', 'n_cafe_cuisine_tea', 'n_cafe_cuisine_dessert',
       'n_cafe_cuisine_bagel', 'n_cafe_cuisine_ice_cream',
       'n_cafe_brand_Tim_Hortons', 'n_cafe_brand_Dutch_Bros._Coffee',
       'n_cafe_cuisine', 'n_cafe_brand'],
      dtype='object')

In [138]:
df_reduced.to_csv('osm_city_cafe_cleaned.csv')

In [140]:
# Identify feature columns (all numeric except city and state)
feature_cols = df.columns.difference(["city_name", "state_code"])
# Rows where all feature columns are zero
df_zero = df[(df[feature_cols] == 0).all(axis=1)]
df_zero['city_name']

31         Albuquerque
54            Honolulu
62             St Paul
68            St Louis
84       St Petersburg
88       Winston Salem
106            Yonkers
113      Port St Lucie
278            Ventura
319        Lees Summit
349          St George
375            OFallon
453       New Rochelle
454              Pharr
496        Cherry Hill
505              Brick
517          St Joseph
539         St Charles
556           St Cloud
579        Schenectady
580         Old Bridge
592    West Bloomfield
658         Piscataway
679       White Plains
689           St Cloud
690    St Clair Shores
692            Margate
712          St Peters
751       Coeur dAlene
806             Delano
811           Rosemead
819      East Honolulu
829      St Louis Park
837     East Brunswick
840            Redford
868         Binghamton
893            Evesham
923         Pearl City
934       Mount Laurel
943             Lompoc
948       Wilkes Barre
951       Oakland Park
953               Hilo
954        